Metadata

In [ ]:
import os
from mutagen.mp4 import MP4, MP4Cover

def change_video_metadata(folder_path, new_description, new_tags):
    """
    특정 폴더 내 모든 동영상 파일의 메타데이터를 변경합니다.

    Args:
        folder_path: 동영상 파일이 있는 폴더 경로
        new_description: 새로 설정할 설명
        new_tags: 새로 설정할 태그 목록
    """

    for filename in os.listdir(folder_path):
        if filename.endswith((".mp4", ".mov", ".m4v")):  # 지원하는 동영상 확장자 추가 가능
            file_path = os.path.join(folder_path, filename)
            try:
                video = MP4(file_path)

                # 설명 변경
                video["\xa9des"] = new_description  # '\xa9des'는 설명을 나타내는 태그

                # 태그 변경 (기존 태그를 모두 지우고 새로 설정)
                video["keyw"] = new_tags  # 'keyw'는 키워드를 나타내는 태그

                video.save()
                print(f"{filename}: 메타데이터 변경 완료")

            except Exception as e:
                print(f"{filename}: 메타데이터 변경 실패 - {e}")

# 사용 예시
folder_path = "/path/to/your/video/folder"  # 동영상 파일이 있는 폴더 경로로 변경
new_description = "새로운 설명입니다."
new_tags = ["태그1", "태그2", "태그3"]

change_video_metadata(folder_path, new_description, new_tags)

In [ ]:
import os
from mutagen.mp4 import MP4, MP4Tags

def update_metadata(directory, description, tags):
    for filename in os.listdir(directory):
        if filename.endswith(".mp4"):
            file_path = os.path.join(directory, filename)
            video = MP4(file_path)
            
            # Ensure MP4Tags exist
            if video.tags is None:
                video.add_tags()
                
            # Update description
            video.tags["desc"] = description
            
            # Update tags
            video.tags["©gen"] = tags
            
            # Save changes
            video.save()
            print(f"Updated metadata for {filename}")

# Usage
directory_path = "/path/to/your/videos"
new_description = "Your new description here"
new_tags = "tag1, tag2, tag3"

update_metadata(directory_path, new_description, new_tags)

Padding

In [ ]:
import os
import numpy as np
import cv2
from moviepy.editor import VideoFileClip, concatenate_videoclips
import multiprocessing
import cupy as cp

def add_padding(frame, padding_size=20):
    """
    프레임에 패딩을 추가합니다. (GPU 가속)

    Args:
        frame: NumPy 배열 형태의 프레임
        padding_size: 추가할 패딩 크기 (픽셀)

    Returns:
        패딩이 추가된 프레임 (NumPy 배열)
    """

    # NumPy 배열을 CuPy 배열로 변환 (GPU로 데이터 전송)
    frame_gpu = cp.asarray(frame)

    # 패딩 추가
    height, width, _ = frame_gpu.shape
    new_height = height + 2 * padding_size
    new_width = width + 2 * padding_size
    padded_frame_gpu = cp.zeros((new_height, new_width, 3), dtype=frame_gpu.dtype)
    padded_frame_gpu[padding_size:padding_size + height, padding_size:padding_size + width] = frame_gpu

    # CuPy 배열을 NumPy 배열로 변환 (CPU로 데이터 다시 가져오기)
    padded_frame = cp.asnumpy(padded_frame_gpu)

    return padded_frame

def process_video(video_path, output_folder, padding_size=20):
    """
    단일 동영상에 패딩을 추가하고 저장합니다.

    Args:
        video_path: 입력 동영상 파일 경로
        output_folder: 출력 동영상을 저장할 폴더 경로
        padding_size: 추가할 패딩 크기 (픽셀)
    """

    try:
        clip = VideoFileClip(video_path)

        # 각 프레임에 패딩 추가 (GPU 가속)
        padded_frames = [add_padding(frame, padding_size) for frame in clip.iter_frames()]

        # MoviePy 클립으로 변환
        padded_clip = concatenate_videoclips([VideoFileClip(lambda t: padded_frames[int(t * clip.fps)], duration=clip.duration)])
        padded_clip = padded_clip.set_fps(clip.fps)

        # 출력 파일 경로 설정
        filename = os.path.basename(video_path)
        output_path = os.path.join(output_folder, f"padded_{filename}")

        # 동영상 저장
        padded_clip.write_videofile(output_path, codec="libx264")  # 필요에 따라 코덱 변경 가능

        print(f"{filename}: 패딩 추가 및 저장 완료")

    except Exception as e:
        print(f"{filename}: 패딩 추가 실패 - {e}")

def add_padding_to_videos_multiprocessing(folder_path, output_folder, padding_size=20):
    """
    폴더 내 모든 동영상에 패딩을 추가합니다 (Multiprocessing 사용).

    Args:
        folder_path: 동영상 파일이 있는 폴더 경로
        output_folder: 출력 동영상을 저장할 폴더 경로
        padding_size: 추가할 패딩 크기 (픽셀)
    """

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)
                   if filename.endswith((".mp4", ".avi", ".mov", ".mkv"))]  # 지원하는 동영상 확장자 추가 가능

    # Multiprocessing을 사용하여 동시 처리
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())  # CPU 코어 수만큼 프로세스 생성
    pool.starmap(process_video, [(video_path, output_folder, padding_size) for video_path in video_files])
    pool.close()
    pool.join()

# 사용 예시
folder_path = "/path/to/your/video/folder"  # 동영상 파일이 있는 폴더 경로로 변경
output_folder = "/path/to/your/output/folder" # 패딩 추가된 동영상을 저장할 폴더 경로로 변경
padding_size = 20

add_padding_to_videos_multiprocessing(folder_path, output_folder, padding_size)